# Draft

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Settings

In [2]:
# # Bitcoin/Thugz League

# TEAMS = 12
# ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

# SKATER_CATEGORIES = {
#     'goals': 7/8, 
#     'assists': 7/8, 
#     'plus_minus': 3/4, 
#     'powerplay_points': 3/4, 
#     'shots_on_goal': 1, 
#     'hits': 1, 
#     'blocks': 1
# }

# GOALIE_CATEGORIES = {
#     "wins" : 1, 
#     "save_percentage": 7/8, 
#     "saves": 3/4, 
#     "shutouts": 1/2
# }

In [3]:
# Yahoo Public Leagues

TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 7/8, 
    'assists': 1, 
    'plus_minus': 3/4, 
    'powerplay_points': 3/4, 
    'shots_on_goal': 1, 
    'hits': 1
}

GOALIE_CATEGORIES = {
    "wins" : 1, 
    "goals_against_average": 7/8,
    "save_percentage": 7/8, 
    "shutouts": 1/2
}

### Players

In [5]:
pic = pd.read_csv("../data/adp-yahoo_com.csv")
cap = pd.read_csv("../data/info-capfriendly_com.csv")
players = pd.merge(pic, cap, how="left", on="name")

players["adp"] = players["pick"] 
players = players[['team', 'age', 'name', 'position', 'adp']]

players.head()

,team,age,name,position,adp
0,Edm,23.0,Connor McDavid,C,1.6
1,Col,25.0,Nathan MacKinnon,C,2.5
2,Edm,25.0,Leon Draisaitl,"C,LW",3.6
3,NYR,29.0,Artemi Panarin,LW,5.2
4,Was,35.0,Alex Ovechkin,LW,5.5


### Multiple Positions

In [6]:
multi = (
    players
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

multi.head()

,name,position
0,Connor McDavid,C
1,Nathan MacKinnon,C
2,Leon Draisaitl,C
3,Leon Draisaitl,LW
4,Artemi Panarin,LW


### Projection Data

In [7]:
proj = pd.read_csv("../data/projections-generated.csv")

### Goalies

In [8]:
goalies = proj[proj["position"] == "G"].copy()

goalies = (
    goalies
    [["name"] + list(GOALIE_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

for key, value in GOALIE_CATEGORIES.items():
    goalies[key] *= value
    
goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= sum(GOALIE_CATEGORIES.values())
goalies["rollup"] *= 100

goalies = goalies.reset_index()
goalies = goalies[["name", "rollup"]]

goalies.head()

,name,rollup
0,Andrei Vasilevskiy,95.315705
1,Connor Hellebuyck,69.446289
2,Tuukka Rask,71.534501
3,Carter Hart,66.108638
4,Robin Lehner,74.598128


### Skaters

In [9]:
skaters = proj[proj["position"] != "G"].copy()
skaters = (
    skaters
    [["name"] + list(SKATER_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)
     
for key, value in SKATER_CATEGORIES.items():
    skaters[key] *= value
    
skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= sum(SKATER_CATEGORIES.values())
skaters["rollup"] *= 100

skaters = skaters.reset_index()
skaters = skaters[["name", "rollup"]]

skaters.head()

,name,rollup
0,Connor McDavid,69.556884
1,Nathan MacKinnon,74.087378
2,Leon Draisaitl,65.928546
3,Artemi Panarin,63.300636
4,Alex Ovechkin,67.336683


### Merge

In [10]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
rollup.head()

,name,position,rollup
7,Andrei Vasilevskiy,G,95.315705
84,Anton Khudobin,G,75.885960
38,Robin Lehner,G,74.598128
1,Nathan MacKinnon,C,74.087378
85,Philipp Grubauer,G,73.053458


### VORP

In [11]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in ROSTER.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * TEAMS)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head()

,name,position,rollup,vorp
7,Andrei Vasilevskiy,G,95.315705,32.998451
84,Anton Khudobin,G,75.885960,13.568706
38,Robin Lehner,G,74.598128,12.280874
1,Nathan MacKinnon,C,74.087378,18.992840
85,Philipp Grubauer,G,73.053458,10.736204


### Draft

In [14]:
draft = pd.merge(players, vorp, how="left", on="name", suffixes=("_yahoo", ""))

draft = draft[[
    'team', 'age', 'name', 
    'position_yahoo', 'position',
    'rollup', 'vorp', 'adp',
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)
draft['pr_vorp'] = draft.groupby("position")['vorp'].rank(ascending=False)
draft['pr_adp'] = draft.groupby("position")['adp'].rank(ascending=True)

draft = draft.sort_values("adp", ascending=True)

In [22]:
draft["pr_arbitrage"] =  draft['pr_adp'] - draft['pr_vorp']
draft['round'] = draft["adp"] // TEAMS + 1

In [23]:
draft.sort_values("pr_arbitrage", ascending=False)

,team,age,name,position_yahoo,position,rollup,vorp,adp,pr_vorp,pr_adp,pr_arbitrage,round
305,Cls,25.0,Oliver Bjorkstrand,"LW,RW",RW,44.8,-6.2,171.5,27.5,72.5,45.0,15.0
336,LA,36.0,Dustin Brown,RW,RW,42.0,-9.0,175.2,39.0,77.0,38.0,15.0
345,Anh,27.0,Rickard Rakell,"LW,RW",RW,40.8,-10.2,178.8,43.0,79.5,36.5,15.0
306,Cls,25.0,Oliver Bjorkstrand,"LW,RW",LW,44.8,-8.9,171.5,31.5,66.0,34.5,15.0
297,Bos,27.0,Matt Grzelcyk,D,D,37.8,-2.2,170.9,28.5,61.5,33.0,15.0
344,Anh,27.0,Rickard Rakell,"LW,RW",LW,40.8,-12.9,178.8,44.0,74.5,30.5,15.0
324,VGK,27.0,William Karlsson,C,C,42.6,-12.5,173.8,34.0,64.0,30.0,15.0
327,Tor,28.0,Zach Hyman,"C,LW",LW,41.8,-12.0,174.4,41.0,71.0,30.0,15.0
328,Tor,28.0,Zach Hyman,"C,LW",C,41.8,-13.3,174.4,35.0,65.0,30.0,15.0
323,Van,34.0,Alexander Edler,D,D,36.0,-4.1,173.6,41.0,70.0,29.0,15.0


In [36]:
pd.DataFrame(draft.groupby(["round"])['position'].value_counts())

position
round position          
1.0   C                6
      LW               3
      G                1
      RW               1
2.0   C                4
      RW               4
      LW               3
      D                2
      G                1
3.0   LW               4
      G                3
      RW               3
      C                2
      D                2
4.0   RW               6
      C                5
      LW               5
      G                2
      D                1
5.0   G                3
      D                2
      LW               2
6.0   C                5
      D                4
      LW               3
      G                2
      RW               2
7.0   G                4
      D                3
      LW               3
      RW               3
      C                2
8.0   D                5
      RW               4
      C                2
      LW               2
      G                1
9.0   D                4
      LW               4
      RW               3
      G                2
      C                1
10.0  D                6
      C                3
      G                2
      LW               2
      RW               1
11.0  D                6
      LW               5
      RW               5
      G                4
      C                2
12.0  C                5
      D                5
      LW               2
      RW               1
13.0  RW               6
      LW               5
      C                4
      D                4
14.0  RW              24
      LW              18
      G               12
      C               10
      D                7
15.0  D               23
      RW              17
      C               16
      LW              14
      G               12
16.0  G                3
      LW               2
      RW               2

### Export

In [21]:
draft.to_csv("../data/draft-yahoo_league.csv", index=False)